In [1]:
import pandas as pd
import requests

# Pull Delhi Metro stations (name/lat/lon) from ArcGIS FeatureServer and print a hard-coded pd.DataFrame block
LAYER_QUERY_URL = (
    "https://services3.arcgis.com/9nfxWATFamVUTTGb/arcgis/rest/services/"
    "Spatial_Accessibility_of_the_Delhi_Metro_Network_WFL1/FeatureServer/1/query"
)

def fetch_all_features(out_fields="Station,Latitude,Longitude", page_size=2000):
    all_feats = []
    offset = 0
    while True:
        params = {
            "where": "1=1",
            "outFields": out_fields,
            "returnGeometry": "false",
            "f": "pjson",
            "resultRecordCount": page_size,
            "resultOffset": offset,
        }
        r = requests.get(LAYER_QUERY_URL, params=params, timeout=60)
        r.raise_for_status()
        data = r.json()
        feats = data.get("features", [])
        if not feats:
            break
        all_feats.extend(feats)
        offset += page_size
        if len(feats) < page_size:
            break
    return all_feats

feats = fetch_all_features()

rows = []
for f in feats:
    a = f.get("attributes", {}) or {}
    rows.append(
        {
            "name": a.get("Station"),
            "lat": a.get("Latitude"),
            "lon": a.get("Longitude"),
        }
    )

metro_stations = pd.DataFrame(rows)
metro_stations["name"] = metro_stations["name"].astype(str).str.strip()
metro_stations["lat"] = pd.to_numeric(metro_stations["lat"], errors="coerce")
metro_stations["lon"] = pd.to_numeric(metro_stations["lon"], errors="coerce")
metro_stations = metro_stations.dropna(subset=["name", "lat", "lon"])

# Deduplicate (some stations appear multiple times); keep one coordinate per station
metro_stations = (
    metro_stations.groupby("name", as_index=False)[["lat", "lon"]]
    .mean()
    .sort_values("name")
    .reset_index(drop=True)
)

# Print a ready-to-paste hardcoded block
names = metro_stations["name"].tolist()
lats = metro_stations["lat"].round(6).tolist()
lons = metro_stations["lon"].round(6).tolist()

print("# --- Paste this block into your script ---")
print("metro_stations = pd.DataFrame({")
print(f"    'name': {repr(names)},")
print(f"    'lat': {repr(lats)},")
print(f"    'lon': {repr(lons)},")
print("})")
print("# --- End block ---")

print(f"\nPulled {len(metro_stations)} unique stations.")


# --- Paste this block into your script ---
metro_stations = pd.DataFrame({
    'name': ['AIIMS', 'Adarsh Nagar', 'Akshardham', 'Anand Vihar ISBT', 'Arjan Garh', 'Arthala', 'Ashok Park Main', 'Ashram', 'Azadpur', 'Badarpur Border', 'Bahadurgarh City', 'Barakhambha Road', 'Bata Chowk', 'Bhikaji Cama Place', 'Botanical Garden', 'Brigadier Hoshiyar Singh', 'Central Secretariat', 'Chandni Chowk', 'Chawri Bazar', 'Chhatarpur', 'Chirag Delhi', 'Civil Lines', 'Dabri Mor-Janakpuri South', 'Dashrath Puri', 'Delhi Aerocity', 'Delhi Cantonment', 'Delhi Gate', 'Dhaula Kuan', 'Dilshad Garden', 'Durgabai Deshmukh South Campus', 'Dwarka', 'Dwarka Mor', 'Dwarka Sector 10', 'Dwarka Sector 11', 'Dwarka Sector 12', 'Dwarka Sector 13', 'Dwarka Sector 14', 'Dwarka Sector 21', 'Dwarka Sector 8', 'Dwarka Sector 9', 'ESI Hospital', 'East Azad Nagar', 'East Vinod Nagar – Mayur Vihar-II', 'Escorts Mujesar', 'GTB Nagar', 'Ghevra', 'Ghitorni', 'Gokulpuri', 'Golf Course', 'Govind Puri', 'Greater Kailash', 'Green P